<div align="left">
    <img src="images/logo_fmkn.png" alt="logo_fmkn" />
</div>

# Python: intro, tips and tricks

<br />
<br />
Александр Авдюшенко <br />
Санкт-Петербург, июнь 2020

![Monty Python](images/monty_python.jpg)

## Язык Python
 * легко [начать использовать](https://colab.research.google.com/)
 * free and open source
 * (почти) portable
 * высокоуровневый
 * интерпретируемый, а не компилируемый
 * REPL = read eval print loop

<div align="center">
    <img src="images/python_after_cpp.jpg" alt="Python after C++" />
</div>

## Пример
$$e^x=\sum_{k=0}^\infty \frac{1}{k!}x^k$$

In [1]:
def e(x):
    sum_, k, term = 1, 0, 1
    while True:
        yield sum_
        k += 1
        term *= x / k
        sum_ += term

In [2]:
x = 1
gen_e_x = e(x)

In [3]:
[next(gen_e_x) for _ in range(5)]

[1, 2.0, 2.5, 2.6666666666666665, 2.708333333333333]

In [4]:
[next(gen_e_x) for _ in range(5)]

[2.7166666666666663,
 2.7180555555555554,
 2.7182539682539684,
 2.71827876984127,
 2.7182815255731922]

## Куайн (квайн, англ. quine) 
Компьютерная программа, которая выдаёт на выходе точную копию своего исходного текста. <br />
При этом программы, использующие внешние данные (чтение текста программы из файла, ввод его с клавиатуры и так далее), куайнами не считаются. Кроме того, не считается куайном «программа», не содержащая вообще никакого кода (вырожденный случай).

In [5]:
_='_=%r;print (_%%_)';print (_%_)

_='_=%r;print (_%%_)';print (_%_)


### Объекты
* **id** — где лежит (~адрес в памяти)
* **type** — какими свойствами обладает
* **value** — значение

In [6]:
id(1), id('1')

(140736941040016, 2708908226480)

In [7]:
type(1), type('1') 

(int, str)

In [8]:
a = 1
b = 1

In [9]:
id(a) == id(1), id(a) == id(b), type(a) == type(1)

(True, True, True)

In [10]:
a, b = 257, 257
id(a) == id(257), id(a) == id(b), type(a) == type(1)

(False, False, True)

In [11]:
a is b # equivalent to id(a) == id(b)

False

## [Size of int in the Python](https://www.quora.com/How-many-bytes-does-an-integer-data-occupy-in-the-Python-language)

Как думаете, сколько памяти занимает int? <br />
Вообще 1 бит это 0 или 1, <br />
1 байт = 8 бит, и это уже от 0 до 255 <br />

Может быть в зависимости от платформы <br />
4 байта (32 бита) или <br />
8 байт (64 бита)?

In [12]:
# давайте проверим!
import sys

def print_sizeof(x):
    return f'{x} — {sys.getsizeof(x)} bytes'

print('\n'.join(['Size of int in Python'] + 
          [print_sizeof(x) for x in (0, 1, 10**10)]))

Size of int in Python
0 — 24 bytes
1 — 28 bytes
10000000000 — 32 bytes


## Почему так много и [не одинаково](https://stackoverflow.com/questions/10365624/sys-getsizeofint-returns-an-unreasonably-large-value)?

In [13]:
isinstance(1, object)

True

[cpython open source realization](https://github.com/python/cpython/blob/ba85d69a3e3610bdd05f0dd372cf4ebca178c7fb/Include/longintrepr.h#L70)
```c++
struct _longobject {
    // macros with
    // 1. the object’s reference count (8 bytes) 
    // 2. and a pointer to the corresponding type object (8 bytes)
    // 3. and extension field ob_size (8 bytes) 
    PyObject_VAR_HEAD 
    // int value adds 0, 4 or 8 bytes 
    digit ob_digit[1]; 
};
```

In [14]:
[method for method in dir(1) if not method.startswith('__')]

['bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes']

In [15]:
import sys

def print_sizeof(x):
    return f'{x} — {sys.getsizeof(x)} bytes'

print('\n'.join(['Size of objects in Python'] + 
          [print_sizeof(x) for x in (0.0, 1.0, "a", "aa", "aaa", 
                                     list(), ["a"], ["a", "aaa"], 
                                     tuple(), ("a",), ("a", "aaa"),
                                     set(), {"a"},  
                                     dict(), {1: "a"},
                                    )]))

Size of objects in Python
0.0 — 24 bytes
1.0 — 24 bytes
a — 50 bytes
aa — 51 bytes
aaa — 52 bytes
[] — 64 bytes
['a'] — 72 bytes
['a', 'aaa'] — 80 bytes
() — 48 bytes
('a',) — 56 bytes
('a', 'aaa') — 64 bytes
set() — 224 bytes
{'a'} — 224 bytes
{} — 240 bytes
{1: 'a'} — 240 bytes


Словари (dict) и множества (set) в Python реализованы как hash-таблицы, [подробности тут](https://stackoverflow.com/questions/327311/how-are-pythons-built-in-dictionaries-implemented).

| index | slot |
| :----: | :---------------: |
| 0 | <hash\|key\|value> |
| 1 | ... |
| . | ... |
| i | ... |
| . | ... |
| n | ... |

 ## Язык Python
 * мультипарадигменный (объектно-ориентированный, функциональный)
 * «батарейки в комплекте» (богатая стандартная библиотека)
 * PEP (python enhanced proposal), [новое в Python 3.8](https://docs.python.org/3/whatsnew/3.8.html)
 * строгая динамическая типизация*

_Тип данных_ — множество значений и операций над этими значениями (IEEE Std 1320.2-1998), их представление в памяти. <br />

Помогает программистам находить ошибки в коде.

In [16]:
5 / 0

ZeroDivisionError: division by zero

Динамическая (утиная) типизация —
"If it looks like a duck, swims like a duck and quacks like a duck, then it probably is a duck."

<div align="center">
    <img src="images/duck.jpg" alt="Duck" width="600" />
</div>

In [17]:
a = 2
print(type(a))
a = True
print(type(a))
# https://en.wikipedia.org/wiki/George_Boole

<class 'int'>
<class 'bool'>


In [18]:
max([1, 2]), max({1, 2})

(2, 2)

Cтрогая (сильная) типизация — наличие безопасности согласования типов и безопасности доступа к памяти. В Python нет (почти) приведения типов.

In [19]:
2 + "1.0"

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [20]:
2 + 1.0

3.0

## Никогда не повторяйте это дома!

In [21]:
def hi(x):
    print("Hello,", x)

def add_two(x):
    return x + 2

add_two.__code__ = hi.__code__

add_two(20)

Hello, 20


In [22]:
import inspect
print(inspect.getsource(hi))

def hi(x):
    print("Hello,", x)



In [23]:
import inspect
print(inspect.getsource(hi))

def hi(x):
    print("Hello,", x)



In [24]:
import dis
dis.dis(hi)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_CONST               1 ('Hello,')
              4 LOAD_FAST                0 (x)
              6 CALL_FUNCTION            2
              8 POP_TOP
             10 LOAD_CONST               0 (None)
             12 RETURN_VALUE


LOAD_GLOBAL    namei — Loads the global named co_names[namei] onto the stack <br />
LOAD_CONST    consti — Pushes "co_consts[consti]" onto the stack <br />
LOAD_FAST    var_num — Pushes a reference to the local co_varnames[var_num] onto the stack

CALL_FUNCTION    argc — Calls a function. The low byte of argc indicates the number of positional parameters, the high byte the number of keyword parameters. On the stack, the opcode finds the keyword parameters first. For each keyword argument, the value is on top of the key. Below the keyword parameters, the positional parameters are on the stack, with the right-most parameter on top. Below the parameters, the function object to call is on the stack.

POP_TOP — Removes the top-of-stack (TOS) item.

## О жизни
 * Python $-$ универсальный клей для API/бибилиотек/фреймворков/распределённых систем
 * Пригождается каждый день
 * Не стоит писать на нём проекты с серьёзной инфраструктурой

С 1991 по 2018 Гвидо ван Россум был «великодушным пожизненным диктатором» языка Python. 

In [25]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


## Python 2 vs Python 3
 * до 1 января 2020 было две официальных, несовместимых версии Python $-$ 2.x и 3.x
 * теперь 2.x $-$ уже **не поддерживается**, никакие новые фичи в нем не появляются, баги не исправляются

In [26]:
import subprocess
result = subprocess.run(['python', '-V'], stdout=subprocess.PIPE)
version = result.stdout.decode('utf-8')
print(f'В презентации используется {version}')

В презентации используется Python 3.7.6



## Ещё примеры

In [27]:
# lazy evaluation
not False or non_existant

True

In [28]:
7 or True

7

In [29]:
1 < 3 < 5

True

In [30]:
False == False != True

True

In [31]:
None

In [32]:
None == None

True

In [33]:
None is None

True

In [34]:
id(None)

140736940563680

In [35]:
a = [1]
b = [1]
a is b # ids

False

In [36]:
a == b # values

True

In [37]:
lst = [1, "Hi!", 3, [1, 2, 3], []]

lst[0] = 4
del lst[1]
lst.insert(1, "Bye!")
lst.append(lst)
print(lst)

[4, 'Bye!', 3, [1, 2, 3], [], [...]]


In [38]:
tpl = (1, 2, 3)
tpl[1] = 2

TypeError: 'tuple' object does not support item assignment

In [39]:
tpl = (1, [2])
tpl[1].append(2)
tpl

(1, [2, 2])

In [40]:
tpl = (1, [2])
tpl[1] += [2]

TypeError: 'tuple' object does not support item assignment

In [41]:
tpl

(1, [2, 2])

### Пример гибкости

In [42]:
import mip
print(f'Using Python-MIP package version {mip.__version__}' )

Using Python-MIP package version 1.8.2


In [43]:
import sys, os
stdout = sys.stdout
sys.stdout = open(os.devnull, 'w')
import mip
print(f'Using Python-MIP package version {mip.__version__}' )
sys.stdout = stdout
print('Ok')

Ok


**Problem 1. Two Sum.** Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

In [44]:
%%writefile two_sum_tests.py

def two_sum(nums, target):
    ans = [0, 1]
    return ans

def test_sorted():
    nums = [2, 7, 11, 15]
    target = 9
    expected = [0, 1]
    output = two_sum(nums, target)
    assert output == expected
    
def test_not_sorted():
    nums = [2, -7, 11, 15]
    target = 4
    expected = [1, 2]
    output = two_sum(nums, target)
    assert output == expected

Writing two_sum_tests.py


In [45]:
!pytest two_sum_tests.py

============================= test session starts =============================
platform win32 -- Python 3.7.6, pytest-5.3.5, py-1.8.1, pluggy-0.13.1
rootdir: C:\
plugins: hypothesis-5.5.4, arraydiff-0.3, astropy-header-0.1.2, doctestplus-0.5.0, openfiles-0.4.0, remotedata-0.3.2
collected 2 items

two_sum_tests.py .F                                                      [100%]

================================== FAILURES ===================================
_______________________________ test_not_sorted _______________________________

    def test_not_sorted():
        nums = [2, -7, 11, 15]
        target = 4
        expected = [1, 2]
        output = two_sum(nums, target)
>       assert output == expected
E       assert [0, 1] == [1, 2]
E         At index 0 diff: 0 != 1
E         Use -v to get the full diff

C:\cygwin64\home\avalur\tech_lectures\two_sum_tests.py:18: AssertionError
========================= 1 failed, 1 passed in 0.10s =========================


**Problem 2. Spiral Matrix II.** Given a positive integer $n$, generate a square matrix filled with elements from 1 to $n^2$ in spiral order.

**Problem 3. Add Two Numbers.** You are given two **non-empty** linked lists representing two non-negative integers. The digits are stored in **reverse order** and each of their nodes contain a single digit. Add the two numbers and return it as a linked list.

You may assume the two numbers do not contain any leading zero, except the number 0 itself.

In [46]:
# Definition for singly-linked list.
class ListNode:
    def __init__(self, x):
        self.val = x
        self.next = None

class Solution:
    def addTwoNumbers(self, l1: ListNode, l2: ListNode) -> ListNode:
        pass        

In [47]:
import this
print(open(this.__file__).read())

s = """Gur Mra bs Clguba, ol Gvz Crgref

Ornhgvshy vf orggre guna htyl.
Rkcyvpvg vf orggre guna vzcyvpvg.
Fvzcyr vf orggre guna pbzcyrk.
Pbzcyrk vf orggre guna pbzcyvpngrq.
Syng vf orggre guna arfgrq.
Fcnefr vf orggre guna qrafr.
Ernqnovyvgl pbhagf.
Fcrpvny pnfrf nera'g fcrpvny rabhtu gb oernx gur ehyrf.
Nygubhtu cenpgvpnyvgl orngf chevgl.
Reebef fubhyq arire cnff fvyragyl.
Hayrff rkcyvpvgyl fvyraprq.
Va gur snpr bs nzovthvgl, ershfr gur grzcgngvba gb thrff.
Gurer fubhyq or bar-- naq cersrenoyl bayl bar --boivbhf jnl gb qb vg.
Nygubhtu gung jnl znl abg or boivbhf ng svefg hayrff lbh'er Qhgpu.
Abj vf orggre guna arire.
Nygubhtu arire vf bsgra orggre guna *evtug* abj.
Vs gur vzcyrzragngvba vf uneq gb rkcynva, vg'f n onq vqrn.
Vs gur vzcyrzragngvba vf rnfl gb rkcynva, vg znl or n tbbq vqrn.
Anzrfcnprf ner bar ubaxvat terng vqrn -- yrg'f qb zber bs gubfr!"""

d = {}
for c in (65, 97):
    for i in range(26):
        d[chr(i+c)] = chr((i+13) % 26 + c)

print("".join([d.get(c, c) for c in s]

Спасибо за внимание, всё на сегодня.